## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [1]:
import os
import re 

import numpy as np
import pandas as pd
from nltk.corpus import stopwords

In [2]:
STOPWORDS = stopwords.words('russian')

In [3]:
main_dir = 'Friends'
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов

for root, dirs, files in os.walk(main_dir):
    if files:
        files_list.extend([os.path.join(root, f) for f in files if not '.DS_Store' in f])

In [4]:
assert len(files_list) == 165

In [4]:
def normalize(doc):
    """
    doc: <str>
    return: <str>
    """
    
    return ' '.join([re.sub('[\W]', '', word.lower()) for word in doc.split() if word.lower() not in STOPWORDS])

def build_doc_collection_from_files(paths, normalization_func=None):
    
    def _read(path):
        """
        return: <str>
        """
        
        with open(path, 'r', encoding='utf-8-sig') as f:  
            return ' '.join(f.readlines()[:-3]) 
        
    if not normalization_func:
        return [_read(path) for path in paths]
    
    else:
        return [normalization_func(_read(path)) for path in paths]
    
def build_word_2_idx_from_collection(collection):
    word2idx = dict()
    _idx = 0
    
    for doc in collection:
        for word in doc.split():
            
            if word not in word2idx:
                word2idx[word] = _idx
                _idx += 1
                
    return word2idx

def build_term_doc_matrix_from_collection(collection, word2idx):
    """
    collection: List[<str>]
    word2idx: <dict>
    
    return: np.array[words x docs]
    """
    
    matrix = np.zeros([len(word2idx), len(collection)])
    
    for i, doc in enumerate(collection):
        for word in doc.split():
            matrix[word2idx[word], i] += 1
            
    return matrix

Нормализованная коллекция

In [5]:
COLLECTION = build_doc_collection_from_files(files_list,
                                             normalization_func=normalize
                                            )

In [7]:
assert len(COLLECTION) == 165

Отображения для индексирования слов и документов

In [6]:
WORD2IDX = build_word_2_idx_from_collection(COLLECTION)
DOC2IDX = {doc: i for i, doc in enumerate(COLLECTION)}
IDX2DOC = {i: doc for i, doc in enumerate(COLLECTION)}

In [14]:
### постройте матрицу терм-документ
term_doc_matrix = build_term_doc_matrix_from_collection(COLLECTION, WORD2IDX)


### напишите функцию булева поиска по построенной матрице

# модуль для парсинга булева выражения
# pip install boolean.py
import boolean


def boolean_search(matrix, query, n=5) -> list:
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents ids
    """
    
    def get_set(arg):
        return set([i for i, x in enumerate(matrix[WORD2IDX[arg.obj.lower()]]) if x > 0])
        
    def _not(_set, x):
        return set(range(matrix.shape[1])).difference(x)
    
    def _and(_set, x):
        return _set.intersection(x)
    
    def _or(_set, x):
        return _set.union(x)
        
    def solve(op):
        q = list()
        
        if 'AND' in str(type(op)):
            op_func = _and
            
        elif 'OR' in str(type(op)):
            op_func = _or
        
        else:
            op_func = _not
        
        # print('OP', op_func.__name__)
        # print('args', op.args)
        
        for arg in op.args:
            if 'Symbol' in str(type(arg)):
                q.append(get_set(arg))
            else:
                q.append(solve(arg))
        
        _set = q[0]
        
        for el in q:
            _set = op_func(_set, el)
        
        # print('returning set', _set)
        
        return _set
    
    query = re.sub('&', 'AND', query)
    query = re.sub('ИЛИ', 'OR', query)
    query = re.sub('НЕ', 'NOT', query)
    
    alg = boolean.BooleanAlgebra()
    exp = alg.parse(query)
    
    if 'Symbol' in str(type(exp)):
        return get_set(exp)
    
    return list(solve(exp))[:n]


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]

Часть матрицы терм-документ в виде таблицы

In [8]:
pd.DataFrame(term_doc_matrix,
             index=list(WORD2IDX.keys()),
             columns=['doc_%s' % i for i, doc in enumerate(COLLECTION)]
            ).head()

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_155,doc_156,doc_157,doc_158,doc_159,doc_160,doc_161,doc_162,doc_163,doc_164
друзья,1.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0
началось,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
нечего,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
рассказывать,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
просто,20.0,5.0,8.0,3.0,11.0,11.0,9.0,9.0,4.0,6.0,...,5.0,6.0,2.0,2.0,3.0,8.0,2.0,3.0,3.0,5.0


Булев поиск по заданным запросам (возвращается список айди релевантных документов)

In [15]:
for query in input_text:
    print('query: %s\nfound: %s\n' % (query, boolean_search(term_doc_matrix, query)))

query: Моника & Фиби & Рэйчел & Чендлер & Джои & Росс
found: [0, 3, 6, 13, 21]

query: (Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс
found: [0, 1, 3, 4, 6]

query: (НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)
found: []



Проверим, например, документ 6 по второму запросу

In [43]:
print(
    ('моника' in COLLECTION[6] or 'фиби' in COLLECTION[6]) and 'рэйчел' in COLLECTION[6] \
    and ('чендлер' in COLLECTION[6] or 'джои' in COLLECTION[6]) and 'росс' in COLLECTION[6]
     )

True


<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [18]:
from collections import defaultdict

In [19]:
def inverted_index(collection) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    inv_idx = defaultdict(dict)
    
    for i, doc in enumerate(collection):
        for word in doc.split():
            
            if inv_idx[word].get(i) is None:
                inv_idx[word][i] = 1
            
            else:
                inv_idx[word][i] += 1
                
    for word in inv_idx.keys():
        inv_idx[word] = [(doc, freq) for doc, freq in inv_idx[word].items()]
                
    return inv_idx

Обратный индекс, включающий частотность по документам вида `(doc_id, freq)` <i>[для выполнения последующих пунктов]</i>

In [20]:
INV_IDX = inverted_index(COLLECTION)

Часть обратного индекса в виде таблицы

In [21]:
INV_IDX_DF = pd.DataFrame.from_dict({word: [docs] for word, docs in INV_IDX.items()}, orient='index')
INV_IDX_DF.columns = ['documents']
INV_IDX_DF.head()

,documents
друзья,"[(0, 1), (2, 3), (4, 2), (10, 1), (11, 1), (12..."
началось,"[(0, 2), (61, 1), (64, 1), (72, 2), (89, 1), (..."
нечего,"[(0, 1), (4, 1), (24, 1), (31, 1), (37, 1), (3..."
рассказывать,"[(0, 1), (1, 1), (18, 1), (32, 1), (45, 1), (4..."
просто,"[(0, 20), (1, 5), (2, 8), (3, 3), (4, 11), (5,..."


С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


1) Общая аналитика

In [22]:
WORD_FREQ_SORTED = sorted(INV_IDX,
                           key=lambda x: sum([freq for doc, freq in INV_IDX[x]]),
                          )

In [23]:
print('Самое частотное слово: %s\nСамое редкое слово: %s\nНабор слов, встречающихся во всех документах: %s' %
        (WORD_FREQ_SORTED[-1],
         WORD_FREQ_SORTED[0],
        [word for word in INV_IDX.keys() if len(INV_IDX[word]) == 165]
        )
     )

Самое частотное слово: это
Самое редкое слово: сотрудник
Набор слов, встречающихся во всех документах: ['просто', 'тебе', 'это', 'да', 'нет', 'что']


2) Частота встречаемости имен главных героев в каждом сезоне

In [26]:
def get_word_freq_per_season(word, inv_idx, print_out=False):
    freqs = dict()
    
    for season in SEASON_MASK.keys():
        freqs[season] = sum([freq for doc, freq in INV_IDX[word] if doc in SEASON_MASK[season]])
        
    if print_out:
        print('"%s"\n\nСезон: Частота' % word)
        
        for season in sorted(freqs, key=freqs.get, reverse=True):
            print('%s: %s' % (season, freqs[season]))
                            
    return freqs


def get_top_freq_word_per_season(words, inv_idx, print_out=False):
    words_freq = dict()
    seasons_freq = dict()
    
    for word in words:
        word_freq = get_word_freq_per_season(word, inv_idx)
        words_freq[word] = word_freq
    
    for season in SEASON_MASK.keys():
        seasons_freq[season] = max([(word, words_freq[word][season]) for word in words], key=lambda x: x[1])
    
    if print_out:
        print('Сезон: Самое частотное слово')
        for season, word in seasons_freq.items():
            print('%s: %s (%s)' % (season, *word))
    
    return seasons_freq

Отображение сезон-документы

In [29]:
SEASON_MASK = defaultdict(list)

for i, f in enumerate(files_list):
    SEASON_MASK[int(re.search('season ([\d])', f).group(1))].append(i)

In [39]:
get_word_freq_per_season('чендлер', INV_IDX, True)

"чендлер"

Сезон: Частота
6: 108
5: 105
7: 91
4: 74
3: 58
1: 48
2: 48


{1: 48, 2: 48, 3: 58, 4: 74, 5: 105, 6: 108, 7: 91}

Самый популярный сезон у Чендлера - 6

In [40]:
get_word_freq_per_season('моника', INV_IDX, True)

"моника"

Сезон: Частота
5: 93
7: 86
6: 82
2: 64
3: 56
1: 46
4: 44


{1: 46, 2: 64, 3: 56, 4: 44, 5: 93, 6: 82, 7: 86}

Самый популярный сезон у Моники - 5

Список чаще всего упоминаемых героев по сезонам

In [31]:
get_top_freq_word_per_season(['чендлер', 'фиби', 'моника', 'росс', 'рэйчел', 'джоуи'], INV_IDX, True)

Сезон: Самое частотное слово
1: рэйчел (67)
2: росс (102)
3: росс (90)
4: джоуи (124)
5: росс (167)
6: росс (140)
7: джоуи (157)


{1: ('рэйчел', 67),
 2: ('росс', 102),
 3: ('росс', 90),
 4: ('джоуи', 124),
 5: ('росс', 167),
 6: ('росс', 140),
 7: ('джоуи', 157)}

Итого, наиболее популярным является Росс, будучи самым часто упоминаемым в 4 сезонах: 2, 3, 5, 6  

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [37]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    
    return log((N - n + 0.5) / (n + 0.5)) * ((k1 + 1) * qf) / (qf + k1 * (1 - b + b * (dl / avgdl)))

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [32]:
avgdl = np.mean([len(doc.split()) for doc in COLLECTION])
N = 165

In [35]:
def compute_sim(query, document_id) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = 0
    dl = len(IDX2DOC[document_id].split())
    
    for word in query.split():
        qf = term_doc_matrix[WORD2IDX[word], document_id]
        n = len(INV_IDX[word])
        score += score_BM25(qf, dl, avgdl, k1, b, N, n)
    
    return score


def get_search_result(query, collection, inv_idx) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    scores = list()
    
    for i, doc in enumerate(collection):
        scores.append((i, compute_sim(query, i)))
        
    return sorted(scores, key=lambda x: x[1], reverse=True)

Выдача по данному запросу

In [38]:
get_search_result('рождественские каникулы', COLLECTION, INV_IDX)[:10]

[(134, 6.666618528089834),
 (29, 6.400140144372682),
 (16, 5.533527536171587),
 (15, 5.450978086258132),
 (150, 3.8397198506472123),
 (70, 3.6740733876352056),
 (124, 3.624984565556023),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0)]

С помощью обратного индекса легко видеть, что не существует документов, в которых встречается вхождение "рождественские каникулы"

In [39]:
INV_IDX['рождественские']

[(15, 2), (16, 2), (29, 3), (70, 1)]

In [40]:
INV_IDX['каникулы']

[(124, 1), (134, 3), (150, 1)]

Из чего следует, что по итогам ранжирования вверху должны оказаться документы, содержащие либо "рождественские", либо "каникулы"<br>
Так и есть, видно, например, что в соответствии с используемой ф-ей ранжирования наиболее релевантен документ, в который "каникулы" входит трижды, а следующим релевантным документом оказывается тот, в который трижды входит "рождественские" 